# Ingest massive amounts of data to a Vector DB (Amazon OpenSearch)
**_Use of Amazon OpenSearch as a vector database for storing embeddings_**

This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Create scripts for ingesting data into OpenSearch](#Create-scripts-for-ingesting-data-into-OpenSearch)
1. [Download the data from the web and upload to S3](#Download-the-data-from-the-web-and-upload-to-S3)
1. [Load the data in a OpenSearch index (Local mode)](#Load-the-data-in-a-OpenSearch-index-(Local-mode))
1. [Load the data in a OpenSearch index via SageMaker Processing Job (Distributed mode)](#Load-the-data-in-a-OpenSearch-index-via-SageMaker-Processing-Job-(Distributed-mode))
1. [Conclusion](#Conclusion)

---

## Objective

This notebook illustrates how to use [`langchain`](https://python.langchain.com/en/latest/index.html) Amazon Sagemaker Endpoints and Amazon Sagemaker Processing Job to convert large amount of data into embeddings and ingest the text data along with its embeddings into an Amazon OpenSearch index.

We use the documents from [sagemaker.readthedocs.io/en/stable](sagemaker.readthedocs.io/en/stable) as the dataset to convert into embeddings. The [`gpt-j-6b`](https://huggingface.co/EleutherAI/gpt-j-6b) large language model (LLM) is to generate the embeddings. 

To understand the code, you might also find it useful to refer to:

- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[SageMaker Processing Job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)*
---

## Background (Problem Description and Approach)

- **Problem statement**: 

Using LLMs for information retrieval tasks (such as question-answering) requires converting the knowledge corpus as well as user questions into vector embeddings. We want to generate these vector embeddings using an LLM hosted as a Amazon Sagemaker Endpoint and store it in a vector database of choice such as Amazon OpenSearch. For converting large amounts of data (TBs or PBs) we need a scalable system which can accomplish both converting the documents into embeddings, storing them in a vector database and provide low latency similarity search

- **Our approach**: 

1. Host the LLM use to generate the embeddings as a SageMaker Endpoint with `instance_count` set to > 1 (the exact number depends upon time taken to generate the embeddings for the amount of data we have and the dollar amount we want to spend on it; more instances would mean greater cost but also lesser time taken).

1. Place the data to be corpus of data in S3 (each document is a file stored as an object in S3).

1. Use a Python script that uses [langchain](https://python.langchain.com/en/latest/index.html) and [Opensearch-py](https://pypi.org/project/opensearch-py/) to ingest the data into OpenSearch. Run the script locally on this notebook for testing.

1. Create a Sagemaker Processing job with `instance_count` set to > 1 (usually matching the `instance_count` for the Sagemaker Endpoint). 

    Each instance of the SageMaker Processing Job runs a script that does the following:
    - Processes a subset of files from S3.
    - Uses langchain to read the files from the local filesystem and convert it into chunks.
    - Creates a langchain `OpenSearchVectorSearch` object and provides it a `SagemakerJumpstartEmbeddings` object that enables it to talk to our Sagemaker Endpoint.
    - Uses the langchain `OpenSearchVectorSearch` to create or get an existing Opensearch index and then ingests documents into the index which contain the original `text`, `embeddings` and `metadata`.
    - Does this using Pytohn multiprocessing to achieve parallelization even within a single processing job instance and ensure maximum use of the Sagemaker Endpoint instance's GPU.
    > **The advantage to using langchain as a wrapper for interfacing with a vector database is that it provides a generic pattern that can be used with any LLM and any vector store. Langchain automatically uses the OpenSearch bulk ingestion API endpoint for ingesting data rather than ingesting data one record at a time. Furthermore, langchain also provides an opinionated JSON structure that includes text and metadata alongwith the embeddings itself for storing embeddings in an OpenSearch index specifically for information retrieval use-cases**.

- **Our tools**: [Amazon SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/), [langchain](https://python.langchain.com/en/latest/index.html) and [Opensearch-py](https://pypi.org/project/opensearch-py/).


---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised by running [this cloud formation template](./template.yaml) before running this notebook.
- A Sagemaker Endpoint for generating embeddings.
- An Amazon OpenSearch cluster for storing embeddings.
    - Opensearch cluster's access credentials (username and password) stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

The overall workflow for this notebook is as follows:
1. Install the required Python packages and store session information in local variables.
1. Download data from source and upload to S3.
1. Run the Python script locally to ingest a subset of data into an OpenSearch index for testing.
1. Run Sagemaker Processing Job which reads all data from S3 and runs the same Python script as above to ingest data into OpenSearch.
    - As part of this step we also create a custom container to package langchain and opensearch Python packages.
1. Do a similarity search with embeddings stored in the OpenSearch index for an input query.

---

## Step 1: Setup
Install the required packages.

In [4]:
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7.0.0 langchain==0.0.149 opensearch-py==2.2.0 faiss_cpu==1.7.4 --quiet
!pip install pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.132 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.0/271.0 kB 4.1 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
import sys
import time
import json
import logging
import numpy as np
from typing import List
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.processing import ProcessingInput
from langchain.document_loaders import ReadTheDocsLoader
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sagemaker.processing import ScriptProcessor, FrameworkProcessor

Change the parameters if you would like to scrape a different website for data, customize chunk size etc.

In [36]:
# global constants
# WEBSITE="https://sagemaker.readthedocs.io/en/stable/"
DOMAIN="doctrine"
DATA_DIR = "docs"
MAX_OS_DOCS_PER_PUT = 1000
IMAGE = "load-data-opensearch-custom"
IMAGE_TAG = "latest"
CHUNK_SIZE_FOR_DOC_SPLIT = 1000
CHUNK_OVERLAP_FOR_DOC_SPLIT = 500
CREATE_OS_INDEX_HINT_FILE = "_create_index_hint"
FAISS_INDEX_DIR = "faiss_index"

In [7]:
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)

### Read parameters from Cloud Formation stack

Some of the resources needed for this notebook such as the Embeddings LLM model endpoint, the Amazon OpenSearch cluster are created outside of this notebook, typically through a cloud formation template. We now read the outputs and parameters of the cloud formation stack created from that template to get the value of these parameters. 

The stack name here should match the stack name you used when creating the cloud formation stack.

In [8]:
# if used a different name while creating the cloud formation stack then change this to match the name you used
CFN_STACK_NAME = "llm-apps-blog-rag2"

**If you did not use a cloud formation template for creating these resources then set the names of these resources manually in the code below.**

In [9]:
#boto3.client('cloudformation').describe_stacks(StackName="ssome")
stacks = boto3.client('cloudformation').list_stacks()
stack_found = CFN_STACK_NAME in [stack['StackName'] for stack in stacks['StackSummaries']]

In [10]:
def get_cfn_outputs(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

def get_cfn_parameters(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    params = {}
    for param in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Parameters']:
        params[param['ParameterKey']] = param['ParameterValue']
    return params

if stack_found is True:
    outputs = get_cfn_outputs(CFN_STACK_NAME)
    params = get_cfn_parameters(CFN_STACK_NAME)
    logger.info(f"cfn outputs={outputs}\nparams={params}")

    embeddings_model_endpoint_name = outputs['EmbeddingEndpointName']
    opensearch_domain_endpoint = f"https://{outputs['OpenSearchDomainEndpoint']}"
    opensearch_index = params['OpenSearchIndexName']
    app_name = params['AppName']
    # ARN of the secret is of the following format arn:aws:secretsmanager:region:account_id:secret:my_path/my_secret_name-autoid
    os_creds_secretid_in_secrets_manager = "-".join(outputs['OpenSearchSecret'].split(":")[-1].split('-')[:-1])
else:
    logger.info(f"cloud formation stack {CFN_STACK_NAME} not found, set parameters manually here")
    # REPLACE THE "placeholder" WITH ACTUAL VALUES IF YOU CREATED THESE RESOURCES WITHOUT USING A CLOUD FORMATION TEMPLATE
    embeddings_model_endpoint_name = "placeholder"
    opensearch_domain_endpoint = "placeholder"
    opensearch_index = "placeholder"
    os_creds_secretid_in_secrets_manager = "placeholder"
    app_name = "llm-apps-blogs"

2023-08-15 08:46:03,018,2617774421,MainProcess,INFO,cfn outputs={'EmbeddingEndpointName': 'llm-apps-blog-gpt-j-6b-endpoint-152d6870', 'OpenSourceDomainArn': 'arn:aws:es:us-east-1:613526579489:domain/opensearchservi-hsbfpnvhbwad', 'OpenSearchDomainEndpoint': 'search-opensearchservi-hsbfpnvhbwad-j3tnhubetgqlqzyncn7hyxvj7u.us-east-1.es.amazonaws.com', 'LLMEndpointName': 'llm-apps-blog-flan-t5-xxl-endpoint-152d6870', 'SageMakerNotebookURL': 'https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/notebook-instances/openNotebook/aws-llm-apps-blog1?view=classic', 'Region': 'us-east-1', 'OpenSearchDomainName': 'opensearchservi-hsbfpnvhbwad', 'LLMAppAPIEndpoint': 'https://jqiv4luk6h.execute-api.us-east-1.amazonaws.com/prod/', 'OpenSearchSecret': 'arn:aws:secretsmanager:us-east-1:613526579489:secret:OpenSearchSecret-llm-apps-blog-rag2-20kDRX'}
params={'SageMakerIAMRole': 'LLMAppsBlogIAMRole', 'SageMakerNotebookName': 'aws-llm-apps-blog1', 'OpenSearchIndexName': 'llm_apps_workshop_embedd

The embeddings model endpoint name, OpenSearch domain endpoint and the identifier for the OpenSearch credentials stored in the Secrets Mananger are all available as `Outputs` from the cloud formation stack.

In [11]:
logger.info(f"embeddings_model_endpoint_name={embeddings_model_endpoint_name},\nopensearch_domain_endpoint={opensearch_domain_endpoint},\n"
            f"os_creds_secretid_in_secrets_manager={os_creds_secretid_in_secrets_manager},opensearch_index={opensearch_index}")

2023-08-15 08:46:03,023,796691109,MainProcess,INFO,embeddings_model_endpoint_name=llm-apps-blog-gpt-j-6b-endpoint-152d6870,
opensearch_domain_endpoint=https://search-opensearchservi-hsbfpnvhbwad-j3tnhubetgqlqzyncn7hyxvj7u.us-east-1.es.amazonaws.com,
os_creds_secretid_in_secrets_manager=OpenSearchSecret-llm-apps-blog-rag2,opensearch_index=llm_apps_workshop_embeddings


In [12]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()
logger.info(f"aws_role={aws_role}, aws_region={aws_region}, bucket={bucket}")

2023-08-15 08:46:11,607,1323976668,MainProcess,INFO,aws_role=arn:aws:iam::613526579489:role/service-role/AmazonSageMaker-ExecutionRole-20221102T161968, aws_region=us-east-1, bucket=sagemaker-us-east-1-613526579489


---

## Step 2: Upload files into the S3 bucket listed above


---

## Step 3: Load data into `OpenSearch`

We are now ready to create scripts which will read data from the local directory, use langchain to create embeddings and then upload the embeddings into OpenSearch.

### Read credentials from AWS Secrets Manager

The credentials for the OpenSearch cluster are store in AWS Secrets Mananger, our code reads the credentials from there and provides them to the opensearch-py package (through langchain API).

### SageMaker embeddings for langchain

langchain provides the [`SagemakerEndpointEmbeddings`]() class which is a wrapper around a functionality to talk to a Sagemaker Endpoint to generate embeddings. We will override the `embed_documents` function to define our own batching strategy for sending requests to the model (multiple requests are sent in one model invocation). Similarly, we extend the `ContentHandlerBase` class to provide implementation for two abstract methods which define how to process (encode/decode) the input data sent to the model and the output received from the model.

We finally create a `SagemakerEndpointEmbeddingsJumpStart` object that puts all this together and can now be used by langchain to talk to an LLM deployed as a Sagemaker Endpoint to generate embeddings.

### Script to load data into OpenSearch

This script puts everything together, it divides the documents into chunks, then uses the langchain package to create embeddings (through `SagemakerEndpointEmbeddingsJumpStart`) and then ingests the data into OpenSearch using `OpenSearchVectorSearch`. 

The langchain `OpenSearchVectorSearch` provides a wrapper over the `opensearch-py` package. It uses the `/_bulk` API endpoint for ingesting multiple records in a single PUT request.

Default chunk size: 1000 tokens.

Default overlap size: 500 tokens. 

---

## Load the data in a [FAISS](https://github.com/facebookresearch/faiss) index (Local mode)

We now create a FAISS index to store the embeddings. This is an alternative to OpenSearch for storing embeddings in-memory. We write the FAISS index locally and then upoad the files to an S3 bucket. A Lambda function can then download these files from S3 and load the FAISS index in memory to perform a similarity search.

In [ ]:
!pip install beautifulsoup4
!pip install xmltodict
!pip install lxml
!pip install pypdf


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [33]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from container.sm_helper import create_sagemaker_embeddings_from_js_model
embeddings = create_sagemaker_embeddings_from_js_model(embeddings_model_endpoint_name, aws_region)

# loader = ReadTheDocsLoader("docs",features="html.parser")
loader = PyPDFLoader("docs/14 OPFOR Tactics.pdf")
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=CHUNK_SIZE_FOR_DOC_SPLIT,
    chunk_overlap=CHUNK_OVERLAP_FOR_DOC_SPLIT,
    # length_function=len,
)
    
# read all the docs, split them into chunks. 
st = time.time() 
logger.info('Loading documents ...')
docs = loader.load()

# add a custom metadata field, such as timestamp
for doc in docs:
    doc.metadata['timestamp'] = time.time()
    doc.metadata['embeddings_model'] = embeddings_model_endpoint_name
chunks = text_splitter.create_documents([doc.page_content for doc in docs], metadatas=[doc.metadata for doc in docs])
et = time.time() - st
logger.info(f'Time taken: {et} seconds. {len(chunks)} chunks generated') 

2023-08-14 18:41:11,053,<ipython-input-33-ee4f77e961b5>,MainProcess,INFO,Loading documents ...
2023-08-14 18:41:39,178,<ipython-input-33-ee4f77e961b5>,MainProcess,INFO,Time taken: 28.124929904937744 seconds. 2474 chunks generated


Load the chunks into FAISS, we provide the embeddings object so that langchain can first convert the text chunks into embeddings and then store those embeddings into a FAISS index.

In [131]:
#temp embedding endpoint
embeddings = create_sagemaker_embeddings_from_js_model("jumpstart-dft-hf-textembedding-gpt-j-6b", aws_region)

vector_db = FAISS.from_documents(chunks, embeddings)

2023-08-01 16:55:41,761,sm_helper,MainProcess,INFO,got results for 2474 in 461.32011461257935s, length of embeddings list is 2474


Save to a local directory and upload to S3.

In [107]:
vector_db_path = FAISS_INDEX_DIR
vector_db.save_local(vector_db_path)

In [108]:
# upload this data to S3, to be used when we run the Sagemaker Processing Job
!aws s3 cp --recursive $vector_db_path/ s3://$bucket/$app_name/$vector_db_path

upload: faiss_index/index.pkl to s3://sagemaker-us-east-1-613526579489/llm-apps-blog/faiss_index/index.pkl
upload: faiss_index/index.faiss to s3://sagemaker-us-east-1-613526579489/llm-apps-blog/faiss_index/index.faiss


---

## Load the data in a `OpenSearch` index via SageMaker Processing Job (Distributed mode)

We now have a working script that is able to ingest data into an OpenSearch index. But for this to work for massive amounts of data we need to scale up the processing by running this code in a distributed fashion. We will do this using Sagemkaer Processing Job. This involves the following steps:

1. Create a custom container in which we will install the `langchain` and `opensearch-py` packges and then upload this container image to Amazon Elastic Container Registry (ECR).
2. Use the Sagemaker `ScriptProcessor` class to create a Sagemaker Processing job that will run on multiple nodes.
    - The data files available in S3 are automatically distributed across in the Sagemaker Processing Job instances by setting `s3_data_distribution_type='ShardedByS3Key'` as part of the `ProcessingInput` provided to the processing job.
    - Each node processes a subset of the files and this brings down the overall time required to ingest the data into Opensearch.
    - Each node also uses Python `multiprocessing` to internally also parallelize the file processing. Thus, **there are two levels of parallelization happening, one at the cluster level where individual nodes are distributing the work (files) amongst themselves and another at the node level where the files in a node are also split between multiple processes running on the node**.

### Create custom container

We will now create a container locally and push the container image to ECR. **The container creation process takes about 1 minute**.

1. The container include all the Python packages we need i.e. `langchain`, `opensearch-py`, `sagemaker` and `beautifulsoup4`.
1. The container also includes the `credentials.py` script for retrieving credentials from Secrets Manager and `sm_helper.py` for helping to create SageMaker endpoint classes that langchain uses.

In [13]:
# Run script to build docker custom containe image and push it to ECR 
# Set region and sagemaker URI variables 
session = boto3.session.Session()
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
logger.info(f"region={aws_region}, account_id={account_id}")
# !bash scripts/build_and_push.sh $(pwd)/container $IMAGE $IMAGE_TAG $aws_region

2023-08-15 08:46:50,495,2352485635,MainProcess,INFO,region=us-east-1, account_id=613526579489


### Create and run the Sagemaker Processing Job

Now we will run the Sagemaker Processing Job to ingest the data into OpenSearch.

In [15]:
embeddings_model_endpoint_name = 'jumpstart-dft-hf-textembedding-gpt-j-6b'

In [51]:
! pip install pdf2image pdfminer pypdf


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [44]:
from langchain.document_loaders import UnstructuredPDFLoader

In [53]:
# setup the parameters for the job
base_job_name = f"{app_name}-job"
tags = [{"Key": "data", "Value": "embeddings-for-llm-apps"}]

# use the custom container we just created
image_uri = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{IMAGE}:{IMAGE_TAG}"

# instance type and count determined via trial and error: how much overall processing time
# and what compute cost works best for your use-case
instance_type = "ml.m5.xlarge"
instance_count = 3
logger.info(f"base_job_name={base_job_name}, tags={tags}, image_uri={image_uri}, instance_type={instance_type}, instance_count={instance_count}")

# setup the ScriptProcessor with the above parameters
processor = ScriptProcessor(base_job_name=base_job_name,
                            image_uri=image_uri,
                            role=aws_role,
                            instance_type=instance_type,
                            instance_count=instance_count,
                            command=["python3"],
                            tags=tags)

# setup input from S3, note the ShardedByS3Key, this ensures that 
# each instance gets a random and equal subset of the files in S3.
inputs = [ProcessingInput(source=f"s3://{bucket}/{app_name}/{DOMAIN}",
                          destination='/opt/ml/processing/input_data',
                          s3_data_distribution_type='ShardedByS3Key',
                          s3_data_type='S3Prefix')]


logger.info(f"creating an opensearch index with name={opensearch_index}")
# ready to run the processing job
st = time.time()
processor.run(code="container/load_data_into_opensearch.py",
              inputs=inputs,
              outputs=[],
              arguments=["--opensearch-cluster-domain", opensearch_domain_endpoint,
                         "--opensearch-secretid", os_creds_secretid_in_secrets_manager,
                         "--opensearch-index-name", opensearch_index,
                         "--aws-region", aws_region,
                         "--embeddings-model-endpoint-name", embeddings_model_endpoint_name,
                         "--chunk-size-for-doc-split", str(CHUNK_SIZE_FOR_DOC_SPLIT),
                         "--chunk-overlap-for-doc-split", str(CHUNK_OVERLAP_FOR_DOC_SPLIT),
                         "--input-data-dir", "/opt/ml/processing/input_data",
                         "--create-index-hint-file", CREATE_OS_INDEX_HINT_FILE,
                         "--process-count", "2"])
time_taken = time.time() - st
logger.info(f"processing job completed, total time taken={time_taken}s")
preprocessing_job_description = processor.jobs[-1].describe()
logger.info(preprocessing_job_description)

2023-08-15 11:42:54,910,2568289917,MainProcess,INFO,base_job_name=llm-apps-blog-job, tags=[{'Key': 'data', 'Value': 'embeddings-for-llm-apps'}], image_uri=613526579489.dkr.ecr.us-east-1.amazonaws.com/load-data-opensearch-custom:latest, instance_type=ml.m5.xlarge, instance_count=3
2023-08-15 11:42:54,933,2568289917,MainProcess,INFO,creating an opensearch index with name=llm_apps_workshop_embeddings
2023-08-15 11:42:55,137,session,MainProcess,INFO,Creating processing-job with name llm-apps-blog-job-2023-08-15-11-42-54-933


.......................2023-08-15 11:46:48,109,load_data_into_opensearch,MainProcess,INFO,Received arguments Namespace(opensearch_cluster_domain='https://search-opensearchservi-hsbfpnvhbwad-j3tnhubetgqlqzyncn7hyxvj7u.us-east-1.es.amazonaws.com', opensearch_secretid='OpenSearchSecret-llm-apps-blog-rag2', opensearch_index_name='llm_apps_workshop_embeddings', aws_region='us-east-1', embeddings_model_endpoint_name='jumpstart-dft-hf-textembedding-gpt-j-6b', chunk_size_for_doc_split=1000, chunk_overlap_for_doc_split=500, input_data_dir='/opt/ml/processing/input_data', process_count=2, create_index_hint_file='_create_index_hint')
2023-08-15 11:46:48,110,load_data_into_opensearch,MainProcess,INFO,there are 9 files to process in the /opt/ml/processing/input_data folder
2023-08-15 11:46:48,262,load_data_into_opensearch,MainProcess,INFO,processing ms_2525d.pdf
Traceback (most recent call last):
  File "/opt/ml/processing/input/code/load_data_into_opensearch.py", line 93, in <module>
    loader = 

UnexpectedStatusException: Error for Processing job llm-apps-blog-job-2023-08-15-11-42-54-933: Failed. Reason: AlgorithmError: See job logs for more information

## Step 4: Do a similarity search for for user input to documents (embeddings) in OpenSearch

In [116]:
from container.credentials import get_credentials
from langchain.vectorstores import OpenSearchVectorSearch
from container.sm_helper import create_sagemaker_embeddings_from_js_model

creds = get_credentials(os_creds_secretid_in_secrets_manager, aws_region)
http_auth = (creds['username'], creds['password'])
docsearch = OpenSearchVectorSearch(index_name=opensearch_index,
                                   embedding_function=create_sagemaker_embeddings_from_js_model("jumpstart-dft-hf-textembedding-gpt-j-6b",
                                                                                                aws_region),
                                   opensearch_url=opensearch_domain_endpoint,
                                   http_auth=http_auth)
q = "What's the proper tactic?"
docs = docsearch.similarity_search(q, k=3) #, search_type="script_scoring", space_type="cosinesimil"
for doc in docs:
    logger.info("----------")
    logger.info(f"content=\"{doc.page_content}\",\nmetadata=\"{doc.metadata}\"")
    

2023-08-01 16:40:48,880,base,MainProcess,WARNING,POST https://search-opensearchservi-hsbfpnvhbwad-j3tnhubetgqlqzyncn7hyxvj7u.us-east-1.es.amazonaws.com:443/%3Clangchain.vectorstores.faiss.FAISS%20object%20at%200x7f898bb9b790%3E/_search [status:404 request:0.051s]


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [langchain.vectorstores.faiss.FAISS object at 0x7f898bb9b790]', langchain.vectorstores.faiss.FAISS object at 0x7f898bb9b790, index_or_alias)

In [114]:
opensearch_domain_endpoint

'https://search-opensearchservi-hsbfpnvhbwad-j3tnhubetgqlqzyncn7hyxvj7u.us-east-1.es.amazonaws.com'

In [117]:
docs

[]

---

## Cleanup

To avoid incurring future charges, delete the resources. You can do this by deleting the CloudFormation template used to create the IAM role and SageMaker notebook.


---

## Conclusion
In this notebook we were able to see how to use LLMs deployed on a SageMaker Endpoint to generate embeddings and then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. We used langchain as an abstraction layer to talk to both the SageMaker Endpoint as well as OpenSearch.

In [140]:
query = "What are the 5 types of combat formations normally used by the tank platoon? Give the answer in a list"
docs = vector_db.similarity_search(query)
print(docs[0].page_content)

or element might happen to be located on the battlefield. However, the function (and hence the functional designation) of a particular force or element may change during the course of the battle. The use of precise functional designations for every force or element on the battlefield allows for a clearer understanding by 
subordinate units of the distinctive functions their co mmander expects them to perform. It also allows each 
force or element to know exactly wh at all of the others are doing at any time. This knowledge facilitates


In [138]:
docs

[Document(page_content='or element might happen to be located on the battlefield. However, the function (and hence the functional designation) of a particular force or element may change during the course of the battle. The use of precise functional designations for every force or element on the battlefield allows for a clearer understanding by \nsubordinate units of the distinctive functions their co mmander expects them to perform. It also allows each \nforce or element to know exactly wh at all of the others are doing at any time. This knowledge facilitates', metadata={'source': 'docs/14 OPFOR Tactics.pdf', 'page': 48, 'timestamp': 1690908480.3316243, 'embeddings_model': 'llm-apps-blog-gpt-j-6b-endpoint-152d6870'}),
 Document(page_content='forces can shift dramatically during the course of a battle, if part of the plan does not work or works better than anticipated. For example, a unit that started out being part of a fixing force might split off and become an exploitation force, if